<a href="https://colab.research.google.com/github/vmali007/Machine-Learning/blob/master/House_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = '/content/train.csv'
file_path1 = '/content/test.csv'
train = pd.read_csv(file_path)
test = pd.read_csv(file_path1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def train_model(X,y,n_estimator):
  train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.1,random_state = 1)
  model = RandomForestRegressor(n_estimators=n_estimator,random_state=1)
  model.fit(train_X,train_y)
  score_ = model.score(test_X,test_y)
  return score_
  
  

In [ ]:
train = train.drop(['Alley','FireplaceQu','PoolQC','Fence','Fence'],axis=1)
test = test.drop(['Alley','FireplaceQu','PoolQC','Fence','Fence'],axis=1)

In [ ]:
X = train.drop('SalePrice',axis=1)
y = train.SalePrice
cat_col = [col for col in X.columns if X[col].dtype == 'object']
numerical_col = [col for col in X.columns if X[col].dtype in ['int64','float64']]
my_col = numerical_col + cat_col

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy ='most_frequent')
X_col = X.columns
test_col = test.columns
X = pd.DataFrame(imputer.fit_transform(X))
test = pd.DataFrame(imputer.transform(test))
X.columns = X_col
test.columns = test_col

In [ ]:
column = []
for col in cat_col:
    if test[col].unique() in X[col].unique():
        column.append(col)
cat_col = column

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in cat_col:
    encoder.fit(X[col])
    X[col] = encoder.transform(X[col])
    test[col] = encoder.transform(test[col])
"""encoder.fit(X[cat_col])
encode = X[cat_col].apply(encoder.transform)
encode_2 = test[cat_col].apply(encoder.transform)"""

'encoder.fit(X[cat_col])\nencode = X[cat_col].apply(encoder.transform)\nencode_2 = test[cat_col].apply(encoder.transform)'

In [ ]:
X = X[numerical_col + cat_col]
test = test[numerical_col + cat_col]
X.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,BldgType,RoofStyle,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,KitchenQual,Functional,GarageType,GarageCond,PavedDrive,SaleType,SaleCondition
0,1,60,65,8450,7,5,2003,2003,196,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003,2,548,0,61,0,0,0,0,0,2,2008,3,1,3,3,0,4,0,2,0,1,1,2,4,2,2,3,3,2,0,1,2,6,1,4,2,8,4
1,2,20,80,9600,6,8,1976,1976,0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976,2,460,298,0,0,0,0,0,0,5,2007,3,1,3,3,0,2,0,1,0,1,2,3,4,1,2,3,1,0,0,1,3,6,1,4,2,8,4
2,3,60,68,11250,7,5,2001,2002,162,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001,2,608,0,42,0,0,0,0,0,9,2008,3,1,0,3,0,4,0,2,0,1,1,2,4,2,2,3,2,2,0,1,2,6,1,4,2,8,4
3,4,70,60,9550,7,5,1915,1970,0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998,3,642,0,35,272,0,0,0,0,2,2006,3,1,0,3,0,0,0,2,0,1,2,3,4,0,3,1,3,0,2,1,2,6,5,4,2,8,0
4,5,60,84,14260,8,5,2000,2000,350,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000,3,836,192,84,0,0,0,0,0,12,2008,3,1,0,3,0,2,0,2,0,1,1,2,4,2,2,3,0,2,0,1,2,6,1,4,2,8,4


In [ ]:
from sklearn.feature_selection import SelectKBest,f_classif

selector = SelectKBest(f_classif,k=50)

X_new = selector.fit_transform(X,y)
X_new

array([[65.0, 8450, 7, ..., 1, 2, 4],
       [80.0, 9600, 6, ..., 1, 2, 4],
       [68.0, 11250, 7, ..., 1, 2, 4],
       ...,
       [66.0, 9042, 7, ..., 1, 2, 4],
       [68.0, 9717, 5, ..., 1, 2, 4],
       [75.0, 9937, 5, ..., 1, 2, 4]], dtype=object)

In [ ]:
selected_features = pd.DataFrame(selector.inverse_transform(X_new),index=X.index,columns=X.columns)

selected_features.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,BldgType,RoofStyle,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,KitchenQual,Functional,GarageType,GarageCond,PavedDrive,SaleType,SaleCondition
0,0,0,65,8450,7,5,2003,2003,196,706,0,150,856,856,854,0,1710,1,0,2,1,3,0,8,0,2003,2,548,0,61,0,0,0,0,0,0,2008,3,1,3,3,0,4,0,0,0,1,1,2,0,2,2,3,3,2,0,1,2,0,1,0,2,0,4
1,0,0,80,9600,6,8,1976,1976,0,978,0,284,1262,1262,0,0,1262,0,0,2,0,3,0,6,1,1976,2,460,298,0,0,0,0,0,0,0,2007,3,1,3,3,0,2,0,0,0,1,2,3,0,1,2,3,1,0,0,1,3,0,1,0,2,0,4
2,0,0,68,11250,7,5,2001,2002,162,486,0,434,920,920,866,0,1786,1,0,2,1,3,0,6,1,2001,2,608,0,42,0,0,0,0,0,0,2008,3,1,0,3,0,4,0,0,0,1,1,2,0,2,2,3,2,2,0,1,2,0,1,0,2,0,4
3,0,0,60,9550,7,5,1915,1970,0,216,0,540,756,961,756,0,1717,1,0,1,0,3,0,7,1,1998,3,642,0,35,0,0,0,0,0,0,2006,3,1,0,3,0,0,0,0,0,1,2,3,0,0,3,1,3,0,2,1,2,0,5,0,2,0,0
4,0,0,84,14260,8,5,2000,2000,350,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,0,9,1,2000,3,836,192,84,0,0,0,0,0,0,2008,3,1,0,3,0,2,0,0,0,1,1,2,0,2,2,3,0,2,0,1,2,0,1,0,2,0,4


In [ ]:
selected_col = selected_features.columns[selected_features.var() != 0]
X[selected_col].head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,ScreenPorch,PoolArea,MiscVal,YrSold,MSZoning,Street,LotShape,LandContour,LotConfig,LandSlope,BldgType,RoofStyle,MasVnrType,ExterQual,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,KitchenQual,GarageType,PavedDrive,SaleCondition
0,65,8450,7,5,2003,2003,196,706,0,150,856,856,854,1710,1,2,1,3,8,0,2003,2,548,0,61,0,0,0,2008,3,1,3,3,4,0,0,1,1,2,2,2,3,3,2,0,1,2,1,2,4
1,80,9600,6,8,1976,1976,0,978,0,284,1262,1262,0,1262,0,2,0,3,6,1,1976,2,460,298,0,0,0,0,2007,3,1,3,3,2,0,0,1,2,3,1,2,3,1,0,0,1,3,1,2,4
2,68,11250,7,5,2001,2002,162,486,0,434,920,920,866,1786,1,2,1,3,6,1,2001,2,608,0,42,0,0,0,2008,3,1,0,3,4,0,0,1,1,2,2,2,3,2,2,0,1,2,1,2,4
3,60,9550,7,5,1915,1970,0,216,0,540,756,961,756,1717,1,1,0,3,7,1,1998,3,642,0,35,0,0,0,2006,3,1,0,3,0,0,0,1,2,3,0,3,1,3,0,2,1,2,5,2,0
4,84,14260,8,5,2000,2000,350,655,0,490,1145,1145,1053,2198,1,2,1,4,9,1,2000,3,836,192,84,0,0,0,2008,3,1,0,3,2,0,0,1,1,2,2,2,3,0,2,0,1,2,1,2,4


In [ ]:
train_model(X[selected_col],y,240)

0.9414919367022258

In [ ]:
model = RandomForestRegressor(n_estimators=240,random_state=1)
model.fit(X[selected_col],y)
y_p = model.predict(test[selected_col])

In [ ]:
df = pd.DataFrame({'Id':test.Id,'SalePrice':y_p})

In [ ]:
df.to_csv('Submition_1.csv',index=False)